# Pràctica 2
### Autors:
<ul>
    <li>Gerard Medina</li>
    <li>Xisco Cerdó</li>
</ul>

## Imports

In [4]:
import os
import shutil
import torch
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

## Classificar el Dataset

In [5]:
# Set the path to the dataset directory
old_dataset = "data/old_pets/images/"

# Set the paths for the output directories
carpeta_moixos = "data/pets/CNN/moix/"
carpeta_cans = "data/pets/CNN/ca/"

# Create the output directories if they don't exist
os.makedirs(carpeta_moixos, exist_ok=True)
os.makedirs(carpeta_cans, exist_ok=True)

# Get the list of all files in the dataset directory
all_files = os.listdir(old_dataset)

# Loop through each file and copy it to the appropriate directory
for filename in all_files:
    full_path = os.path.join(old_dataset, filename)
    if filename[0].islower():
        shutil.copy(full_path, os.path.join(carpeta_cans, filename))
    elif filename[0].isupper():
        shutil.copy(full_path, os.path.join(carpeta_moixos, filename))

## Carregar el Dataset

In [6]:
new_dataset = "data/pets/CNN/"

# Define the transformations to be applied to the dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = datasets.ImageFolder(root=new_dataset, transform=transform)

# Define the data loader
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

## Mostrar exemples nou Dataset

In [ ]:
# Print the length of the dataset
print("Number of images in dataset: ", len(dataset))

# Show some sample images
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img_idx = i*100
    img, label = dataset[img_idx]
    fig.add_subplot(rows, columns, i)
    plt.title("Label: "+dataset.classes[label])
    plt.imshow(img.permute(1, 2, 0))
plt.show()